<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/graph_state/StateValidationWithPydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Example of Private State


## Goals

we're going to illustrate how to use private state and private schema to communicate between internal nodes.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph

So we define three containers all of which subclass from `TypedDict` - `OverallState`, `Node1Input`, and `Node2Input`.

In [2]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# The overall state of the graph (this is the public state shared across nodes)
class OverallState(TypedDict):
    a: str

# Output from `node_1` contains private data that is not part of the overall state
class Node1Output(TypedDict):
    private_data: str

# The input to Node 2 contains the private data available after `node_1` completes execution
class Node2Input(TypedDict):
    private_data: str

We wire the private state - `Node1Output` as `node_1` output and `Node2Input` as the input to `node_2` with the following code

In [3]:
# The private data is only shared between `node_1` and `node_2`
def node_1(state: OverallState) -> Node1Output:
    output = {"private_data": "set by node_1"}
    print(f"Entered node `node_1`:\n\tInput: {state}.\n\tReturned: {output}")
    return output

def node_2(state: Node2Input) -> OverallState:
    output = {"a": "set by node_2"}
    print(f"Entered node `node_2`:\n\tInput: {state}.\n\tReturned: {output}")
    return output

We wire the overall (public) state with both the input and output of `node_3`

In [4]:
# Node 3 only has access to the overall state (no access to private data from `node_1`)
def node_3(state: OverallState) -> OverallState:
    output = {"a": "set by node_3"}
    print(f"Entered node `node_3`:\n\tInput: {state}.\n\tReturned: {output}")
    return output

We connect all nodes in a sequence, where `node_2` acceppts private data from `node_1`, and `node_3` does not see the private data at all. Then we execute the graph.

In [5]:
builder = StateGraph(OverallState).add_sequence([node_1, node_2, node_3])
builder.add_edge(START, "node_1")
graph = builder.compile()

# Invoke the graph with the initial state
response = graph.invoke(
    {
        "a": "set at start",
    }
)
print()
print(f"Output of graph invocation: {response}")

Entered node `node_1`:
	Input: {'a': 'set at start'}.
	Returned: {'private_data': 'set by node_1'}
Entered node `node_2`:
	Input: {'private_data': 'set by node_1'}.
	Returned: {'a': 'set by node_2'}
Entered node `node_3`:
	Input: {'a': 'set by node_2'}.
	Returned: {'a': 'set by node_3'}

Output of graph invocation: {'a': 'set by node_3'}
